In [1]:
from pyspark.sql import SparkSession


import configparser

# Tạo một parser
config = configparser.ConfigParser()

# Đọc file config
config.read('secret.ini')
access_key = config['minio']['access_key']
secret_key = config['minio']['secret_key']

spark = SparkSession.builder\
             .appName("test")\
             .master("local[2]") \
             .config("spark.driver.extraJavaOptions",'-Dhttp.proxyHost=10.60.117.103 -Dhttp.proxyPort=8085 -Dhttps.proxyHost=10.60.117.103 -Dhttps.proxyPort=8085 -Dhttp.nonProxyHosts="localhost|127.0.0.1|10.208.164.167"') \
             .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.1,io.delta:delta-core_2.12:2.1.0") \
             .config("spark.hadoop.fs.s3a.access.key", access_key)\
             .config("spark.hadoop.fs.s3a.secret.key", secret_key)\
             .config("spark.hadoop.fs.s3a.endpoint", "http://10.208.164.167:9000") \
             .config("spark.hadoop.fs.s3a.path.style.access", "true")\
             .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
             .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")\
             .getOrCreate()

#              .config("fs.s3a.connection.ssl.enabled", "false") \
#              .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \

spark.sparkContext.setLogLevel("ERROR")

# chạy jar chắc vẫn bị thiếu depandency  -> chueyern sang chạy pack để tải cho đủ, nhưng mà  pahri ignore proxy, nếu không nó bị treo luôn
#            .config("spark.jars", "/home/hadoop-aws-3.3.1.jar,/home/delta-core_2.12-2.1.0.jar") \

# cái này chạy được nè
             # .config("spark.driver.extraJavaOptions","-Dhttp.proxyHost=10.60.117.103 -Dhttp.proxyPort=8085 -Dhttps.proxyHost=10.60.117.103 -Dhttps.proxyPort=8085") \
             # .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.1,io.delta:delta-core_2.12:2.1.0") \

# lỗi Py4JJavaError: An error occurred while calling o53.load.
# : java.lang.IncompatibleClassChangeError: class org.apache.spark.sql.catalyst.plans.logical.DeltaDelete has interface org.apache.spark.sql.catalyst.plans.logical.UnaryNode as super class
             # .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
             # .config("spark.jars", "/home/hadoop-aws-3.2.0.jar") \



#              .config("spark.jars", "/home/delta-core_2.12-1.0.0.jar") \
#/home/aws-java-sdk-bundle-1.11.375.jar,/home/hadoop-aws-3.2.0.jar,
             # .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0")
             #.config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0, org.apache.hadoop:hadoop-aws:3.2.0, com.amazonaws:aws-java-sdk-bundle:1.11.375")

:: loading settings :: url = jar:file:/usr/local/lib/python3.8/dist-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-15b34181-26d1-48f2-9fda-f23ae7435a1f;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.1 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.901 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
	found io.delta#delta-core_2.12;2.1.0 in central
	found io.delta#delta-storage;2.1.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
downloading https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-aws/3.3.1/hadoop-aws-3.3.1.jar ...
	[SUCCESSFUL ] org.apache.hadoop#hadoop-aws;3.3.1!hadoop-aws.jar (968ms)
downloading https://repo1.maven.org/maven2/io/delta/delta-core_2.12/2.1.0/delta-core_2.12-2.1.0.jar ...
	[SUCCESSFUL 

25/01/04 03:52:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
data_csv = '/home/data.csv'
data_df = spark.read.format('csv').option('header', 'true').option('inferSchema', 'true').load(data_csv)

In [3]:
data_df

DataFrame[year: int, age: int, school: int, group: int, topic: int, count: string]

In [4]:
data_df.show(10)

+----+---+------+-----+------+-----+
|year|age|school|group| topic|count|
+----+---+------+-----+------+-----+
|2018| 10|     2|    9|133500|    0|
|2018| 10|     3|    1|133500|  ..C|
|2018| 10|     3|    2|133500|  ..C|
|2018| 10|     3|    9|133500|    0|
|2018| 10|     4|    1|133500|  ..C|
|2018| 10|     4|    2|133500|  ..C|
|2018| 10|     4|    9|133500|    3|
|2018| 10|     5|    1|133500|  ..C|
|2018| 10|     5|    2|133500|  ..C|
|2018| 10|     5|    9|133500|    0|
+----+---+------+-----+------+-----+
only showing top 10 rows



In [7]:
S3_BUCKET= "delta-demo"
delta_table_name = "table"

In [8]:
data_df.write.format("delta").save(f"s3a://{S3_BUCKET}/{delta_table_name}", mode="overwrite")

In [9]:
from delta import *
minio_bucket = "delta-demo-1"
data_df \
    .write \
    .format("delta") \
    .partitionBy("school") \
    .mode("overwrite") \
    .save(f"s3a://{minio_bucket}/data-delta")

In [2]:
from delta import *
minio_bucket = "delta-demo-1"
df = spark \
    .read \
    .format("delta") \
    .load(f"s3a://{minio_bucket}/data-delta")

In [3]:
df.show()

+----+---+------+-----+------+-----+
|year|age|school|group| topic|count|
+----+---+------+-----+------+-----+
|2018| 10|     3|    1|133500|  ..C|
|2018| 10|     3|    2|133500|  ..C|
|2018| 10|     3|    9|133500|    0|
|2018| 10|     4|    1|133500|  ..C|
|2018| 10|     4|    2|133500|  ..C|
|2018| 10|     4|    9|133500|    3|
|2018| 10|     5|    1|133500|  ..C|
|2018| 10|     5|    2|133500|  ..C|
|2018| 10|     5|    9|133500|    0|
|2018| 10|     6|    1|133500|  ..C|
|2018| 10|     2|    9|133500|    0|
+----+---+------+-----+------+-----+



In [6]:
# MERGE EXAMPLE
target_table_path = "s3a://delta-demo-5/data-delta"

# Example target Delta table (if not created)
target_data = [
    (1, "John Doe", 25),
    (2, "Jane Smith", 30)
]
target_columns = ["id", "name", "age"]

target_df = spark.createDataFrame(target_data, target_columns)
target_df.write.format("delta").mode("overwrite").save(target_table_path)

# Load source data
source_data = [
    (1, "John Doe", 26),  # Updated age for John Doe
    (3, "Alice Johnson", 22)  # New record for Alice
]
source_columns = ["id", "name", "age"]

source_df = spark.createDataFrame(source_data, source_columns)

# Perform the Delta Lake MERGE operation
delta_table = DeltaTable.forPath(spark, target_table_path)

delta_table.alias("target").merge(
    source=source_df.alias("source"),
    condition="target.id = source.id"
).whenMatchedUpdate(set={
    "name": "source.name",
    "age": "source.age"
}).whenNotMatchedInsert(values={
    "id": "source.id",
    "name": "source.name",
    "age": "source.age"
}).execute()

# Show the updated target Delta table
updated_df = spark.read.format("delta").load(target_table_path)
updated_df.show()

Py4JJavaError: An error occurred while calling o146.save.
: java.util.concurrent.ExecutionException: org.apache.hadoop.fs.s3a.UnknownStoreException: s3a://delta-demo-5/data-delta/_delta_log
	at com.google.common.util.concurrent.AbstractFuture$Sync.getValue(AbstractFuture.java:306)
	at com.google.common.util.concurrent.AbstractFuture$Sync.get(AbstractFuture.java:293)
	at com.google.common.util.concurrent.AbstractFuture.get(AbstractFuture.java:116)
	at com.google.common.util.concurrent.Uninterruptibles.getUninterruptibly(Uninterruptibles.java:135)
	at com.google.common.cache.LocalCache$Segment.getAndRecordStats(LocalCache.java:2410)
	at com.google.common.cache.LocalCache$Segment.loadSync(LocalCache.java:2380)
	at com.google.common.cache.LocalCache$Segment.lockedGetOrLoad(LocalCache.java:2342)
	at com.google.common.cache.LocalCache$Segment.get(LocalCache.java:2257)
	at com.google.common.cache.LocalCache.get(LocalCache.java:4000)
	at com.google.common.cache.LocalCache$LocalManualCache.get(